# **Environment Configuration**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Missing value marks
true_missing = -9
derived_missing = -99

## Load Data

In [ ]:
# Load indigent burial data
#data = pd.read_csv("./data/indigent_burials.csv")
file_path = input("Path to CSV file: ")
data = pd.read_csv(file_path)

# **To Do Cleaning Tasks**
----

## Look into Location variable

In [ ]:
data.Location.value_counts()

## Date of Birth

In [ ]:
data.DOB.sort_values(ascending=True)

## Clean DOD

If the death date is greater than 2022, subtract 100 from the year.

# **Save Cleaned CSV File**
----

In [ ]:
file_name = input("File name: ")
data.to_csv('./data/' + file_name + '.csv', index = False)

# **Completed Cleaning Tasks**
----

## Append new Hart Island data to dataframe

In [ ]:
# Load Hart Island Data
hart_island = pd.read_csv("https://raw.githubusercontent.com/IndigentBurials/hart-island-web-scraper/main/data/output/hart-island.csv")

In [ ]:
# Drop all old Hart Island entries
data = data[data['Jurisdiction'] != "Hart Island"]

In [ ]:
# Combine add new hart island data back in
data = pd.concat([data, hart_island])
# Reset the index
data = data.reset_index().drop(columns=['index'])

## Standardize Sex column to M/F/U

In [ ]:
# Standardize column values
data['Sex'] = data['Sex'].str.upper()

# Get list of aliases from data
male_alias = ['MALE', 'M']
female_alias = ['FEMALE', 'F']
unknown_alias = ['UNKNOWN', 'UNSURE', '--']

# Create list for sex
sex_list = []

for index, row in data.iterrows():
    if row['Sex'] in male_alias:
        sex_list.append("M")
    elif row['Sex'] in female_alias:
        sex_list.append("F")
    else:
        sex_list.append("U")
        
data['Sex'] = sex_list

## Drop Age Group Column

In [ ]:
data = data.drop(columns=['AgeGroup'])

## Standardize Age variable

In [ ]:
ages = list(data.Age)
new_ages = []

for age in ages:
    # Try to cast to an integer
    try:
        # If we can cast the age to an integer, we append it here
        t_age = int(age)
        if t_age > 150:
            new_ages.append(true_missing)
        else:
            new_ages.append(age)
    except:
        # If the value is missing, we replace it with the true missing value
        if str(age) == "nan":
            new_ages.append(true_missing)
        # If it fails, there are underlying strings. This is indicative
        # of a less-than-1-year-old, so we denote them as '0'.
        else:
            new_ages.append(0)

# Reassign the column
data['Age'] = new_ages

## Create flag for gather type

In [ ]:
data['GatherType'] = 'Web Scraping'

## Fill NaN values with True/Derived missing values

In [ ]:
data['LName'] = data['LName'].fillna(true_missing)
data['FName'] = data['FName'].fillna(true_missing)
data['MName'] = data['MName'].fillna(true_missing)
data['OName'] = data['OName'].fillna(true_missing)
data['Sex'] = data['Sex'].fillna(true_missing)
data['RaceEthincity'] = data['RaceEthincity'].fillna(true_missing)
data['DOB'] = data['DOB'].fillna(true_missing)
data['DOD'] = data['DOD'].fillna(true_missing)
data['DeathYear'] = data['DeathYear'].fillna(derived_missing)
data['DBF'] = data['DBF'].fillna(true_missing)
data['Mortuary'] = data['Mortuary'].fillna(true_missing)
data['FuneralDirector'] = data['FuneralDirector'].fillna(true_missing)
data['CaseNo'] = data['CaseNo'].fillna(true_missing)
data['PD'] = data['PD'].fillna(true_missing)
data['BD'] = data['BD'].fillna(true_missing)
data['BurialYear'] = data['BurialYear'].fillna(derived_missing)
data['CremationNo'] = data['CremationNo'].fillna(true_missing)
data['GraveSection'] = data['GraveSection'].fillna(true_missing)
data['GraveLot'] = data['GraveLot'].fillna(true_missing)
data['GraveNo'] = data['GraveNo'].fillna(true_missing)
data['Age'] = data['Age'].fillna(derived_missing)
data['Jurisdiction'] = data['Jurisdiction'].fillna(true_missing)
data['State'] = data['State'].fillna(derived_missing)
data['County'] = data['County'].fillna(true_missing)
data['CountyNo'] = data['CountyNo'].fillna(true_missing)
data['City'] = data['City'].fillna(true_missing)
data['Location'] = data['Location'].fillna(true_missing)
data['CityOfBirth'] = data['CityOfBirth'].fillna(true_missing)
data['StateOfBirth'] = data['StateOfBirth'].fillna(true_missing)
data['SourceURL'] = data['SourceURL'].fillna(derived_missing)
data['DateScraped'] = data['DateScraped'].fillna(derived_missing)
data['LastModified'] = data['LastModified'].fillna(true_missing)
data['Misc'] = data['Misc'].fillna(true_missing)
data['isVeteran'] = data['isVeteran'].fillna(derived_missing)
data['MilitaryAffiliation'] = data['MilitaryAffiliation'].fillna(true_missing)
data['POD'] = data['POD'].fillna(true_missing)
data['PlotNo'] = data['PlotNo'].fillna(true_missing)
data['GatherType'] = data['GatherType'].fillna(derived_missing)

## Fix DeathYear Column

In [ ]:
dods = []
for dod in list(data.DOD):
    try:
        t_date = pd.to_datetime(dod)
        dods.append(t_date)
    except:
        dods.append(true_missing)
        continue

# Get death year
death_years = []
for dod in dods:
    if dod == true_missing:
        death_years.append(derived_missing)
    else:
        death_years.append(dod.year)

data.DeathYear = death_years

## Drop isClaimed flag

In [ ]:
data = data.drop(columns=['isClaimed'])